In [28]:
import numpy as np

def spherical_tile_area(tile_size_deg, dec_center_deg):
    """
    Computes the spherical area of a rectangular patch on the sky
    defined by:
      - RA interval: tile_size_deg
      - Dec interval: tile_size_deg
    centered at dec_center_deg.
    
    The spherical area is computed on the unit sphere using:
    
        area_sr = ΔRA (in rad) * [ sin(dec_center + tile_size_deg/2 in rad) 
                                  - sin(dec_center - tile_size_deg/2 in rad) ]
                                  
    Then converted to square degrees. 

    Parameters:
      tile_size_deg : float
         Side length of the square tile, in degrees.
      dec_center_deg : float
         Central declination (in degrees) where the tile is located.

    Returns:
      area_sph_deg2 : float
         The area computed using spherical geometry, in square degrees.
    """
    # Convert tile size and dec_center to radians
    L_rad = np.radians(tile_size_deg)
    dec_center_rad = np.radians(dec_center_deg)
    
    # Compute the declination boundaries
    dec1_rad = dec_center_rad - L_rad / 2.0
    dec2_rad = dec_center_rad + L_rad / 2.0
    
    # Spherical area in steradians. Note that for a patch defined by RA and Dec,
    # the area is: (ΔRA in rad) * (sin(dec2) - sin(dec1))
    area_sr = L_rad * (np.sin(dec2_rad) - np.sin(dec1_rad))
    
    # Convert steradians to square degrees:
    #  1 sr = (180/π)² deg².
    area_sph_deg2 = area_sr * (180.0 / np.pi) ** 2
    return area_sph_deg2

def compare_tile_areas(dec_center_deg):
    """
    Compares the spherical area vs. the naive (flat) area 
    for a 10°×10° tile and a 2°×2° tile, at a given declination.

    The naive area is simply tile_size^2.
    """
    for tile_size in [10, 2]:
        naive_area = tile_size ** 2
        actual_area = spherical_tile_area(tile_size, dec_center_deg)
        diff = naive_area - actual_area
        percent_diff = (diff / naive_area) * 100.0

        print(f"Tile size: {tile_size}° × {tile_size}° at dec_center = {dec_center_deg}°")
        print(f"  Naive area (deg²): {naive_area:.2f}")
        print(f"  Spherical area (deg²): {actual_area:.2f}")
        print(f"  Difference: {diff:.2f} deg² ({percent_diff:.1f}% less)\n")

# Example usage:
if __name__ == '__main__':
    # Change the central declination to see different behavior.
    # For example, dec_center_deg = 0 (equator) versus dec_center_deg = 60.
    print("Comparison at dec_center = 0° (low declination):")
    compare_tile_areas(dec_center_deg=0)

    print("Comparison at dec_center = 60° (high declination):")
    compare_tile_areas(dec_center_deg=60)


Comparison at dec_center = 0° (low declination):
Tile size: 10° × 10° at dec_center = 0°
  Naive area (deg²): 100.00
  Spherical area (deg²): 99.87
  Difference: 0.13 deg² (0.1% less)

Tile size: 2° × 2° at dec_center = 0°
  Naive area (deg²): 4.00
  Spherical area (deg²): 4.00
  Difference: 0.00 deg² (0.0% less)

Comparison at dec_center = 60° (high declination):
Tile size: 10° × 10° at dec_center = 60°
  Naive area (deg²): 100.00
  Spherical area (deg²): 49.94
  Difference: 50.06 deg² (50.1% less)

Tile size: 2° × 2° at dec_center = 60°
  Naive area (deg²): 4.00
  Spherical area (deg²): 2.00
  Difference: 2.00 deg² (50.0% less)



In [48]:
from astropy.coordinates import SkyCoord
import astropy.units as u

# Define the coordinates in degrees
# Example coordinates
ra1, dec1 = 100, 0  # RA and Dec for point 1 (e.g., Andromeda Galaxy)
ra2, dec2 = 100 + 100*10/3600, 0  # RA and Dec for point 2 (e.g., Orion Nebula)

# Create SkyCoord objects
coord1 = SkyCoord(ra=ra1 * u.degree, dec=dec1 * u.degree, frame='icrs')
coord2 = SkyCoord(ra=ra2 * u.degree, dec=dec2 * u.degree, frame='icrs')

# Calculate the angular separation
separation = coord1.separation(coord2)

# Print the result
print(f"Angular separation: {separation.degree:.4f} degrees")


Angular separation: 0.2778 degrees


In [44]:
from astropy.wcs import WCS
import numpy as np

def pixel_ra_difference(crval_dec):
    # Create a WCS with a TAN projection.
    w = WCS(naxis=2)
    w.wcs.crpix = [1000, 1000]            # Reference pixel (center)
    cd = 10.0 / 3600.0                    # 10 arcsec in degrees
    w.wcs.cdelt = np.array([-cd, cd])     # Pixel scale (negative for RA axis convention)
    w.wcs.crval = [10.0, crval_dec]         # Reference world coordinates (RA, DEC)
    w.wcs.ctype = ["RA---TAN", "DEC--TAN"]  # TAN (gnomonic) projection
    
    # Calculate the world coordinates for two pixels that differ by 1 in the X direction.
    pos1 = w.all_pix2world([[1000, 1000]], 1)[0]
    pos2 = w.all_pix2world([[1100, 1000]], 1)[0]

    # Compute the difference in RA (in degrees) and convert to arcseconds.
    dRA_deg = pos2[0] - pos1[0]
    dRA_arcsec = dRA_deg * 3600.0
    return pos1, pos2, dRA_arcsec

# Calculate for a tangent plane centered at DEC = 0°.
pos1_0, pos2_0, dRA_arcsec_0 = pixel_ra_difference(0)
print("Tangent point at DEC = 0°:")
print("Pixel (1000, 1000): RA, DEC =", pos1_0)
print("Pixel (1001, 1000): RA, DEC =", pos2_0)
print(f"Difference in RA: {dRA_arcsec_0:.2f} arcsec\n")

# Calculate for a tangent plane centered at DEC = 50°.
pos1_50, pos2_50, dRA_arcsec_50 = pixel_ra_difference(50)
print("Tangent point at DEC = 50°:")
print("Pixel (1000, 1000): RA, DEC =", pos1_50)
print("Pixel (1001, 1000): RA, DEC =", pos2_50)
print(f"Difference in RA: {dRA_arcsec_50:.2f} arcsec")


Tangent point at DEC = 0°:
Pixel (1000, 1000): RA, DEC = [10.  0.]
Pixel (1001, 1000): RA, DEC = [9.72222440e+00 3.45728707e-15]
Difference in RA: -999.99 arcsec

Tangent point at DEC = 50°:
Pixel (1000, 1000): RA, DEC = [10. 50.]
Pixel (1001, 1000): RA, DEC = [ 9.56786269 49.99919755]
Difference in RA: -1555.69 arcsec


In [49]:
dRA_arcsec_0/dRA_arcsec_50

0.6427947621591361

In [50]:
0.2778*0.6427947621591361

0.178568384927808